In [ ]:
import argparse
from os import makedirs, listdir
from fsbi.utils import save_data_to_hdf5, read_monitor_weights_files, _forward
import numpy as np
import yaml
import h5py
import matplotlib

import pickle
from concurrent import futures
import matplotlib.pyplot as plt
import torch
import time

#### 1/ Simulation parameters, hardware options for polynomial rules

In [ ]:
save_dir = "data_synapsesbi/bg_IF_EEEIIEII_6pPol/"
runs_path = "runs_synapsesbi/bg_IF_EEEIIEII_6pPol/"
task_name = "bg_IF_EEEIIEII_6pPol"
round_name = "pi3_r5to10Hz"
from fsbi.simulator import Simulator_bg_IF_EEEIIEII_6pPol as Simulator

#For parallel execution on local computer
hardware = "local"
parallel_args = dict()
parallel_args["max_workers"]= 10 #depending on how many cpus are available

#Get simulator params and make simulator
with open("tasks_configs/%s.yaml" % task_name, "r") as f:
    simulator_params = yaml.load(f, Loader=yaml.Loader)
simulator = Simulator(simulator_params)

#### 1 bis/ Simulation parameters, hardware options for MLP rules

In [ ]:
save_dir = "data_synapsesbi/bg_CVAIF_EEIE_T4wvceciMLP/"
runs_path = "runs_synapsesbi/bg_CVAIF_EEIE_T4wvceciMLP/"
task_name = "bg_CVAIF_EEIE_T4wvceciMLP"
round_name = "pi3_r5to10Hz"
from synapsbi.simulator import Simulator_bg_CVAIF_EEIE_T4wvceciMLP as Simulator

#For parallel execution on local computer
hardware = "local"
parallel_args = dict()
parallel_args["max_workers"]= 10 #depending on how many cpus are available

#Get simulator params and make simulator
with open("tasks_configs/%s.yaml" % task_name, "r") as f:
    simulator_params = yaml.load(f, Loader=yaml.Loader)
simulator = Simulator(simulator_params)

#### Use samples generated in previous notebook

In [ ]:
thetas = np.load(save_dir + "post_samples_" + round_name + ".npz")["thetas"]
num_samples = len(thetas)

thetas, seeds = _make_unique_samples(num_samples, 
                                     prior=None,
                                     thetas=thetas,
                                     saved_seeds=[])

#### Simulate the samples

Refer to readme to make sure the spiking network simulator Auryn is installed on your machine

In [ ]:
### prepare thetas for the simulation
parameters = [dict(simulator=simulator,
                   thetas=th.reshape(1, -1),
                   seeds=[seed],
                   return_data=False)
              for th, seed in zip(thetas, seeds)][:num_samples]

In [ ]:
## Quick look at the samples
try:
    a = np.array([i['thetas'].numpy() for i in parameters])
except:
    a = np.array([i['thetas'] for i in parameters])
print(a.shape)

fig = plt.figure(figsize=(3, 2)), plt.scatter(a[:,0,2],a[:,0,3], s = 1); plt.show()

In [ ]:
start = time.time()
if hardware == "local":
    with futures.ThreadPoolExecutor(**parallel_args) as executor:
        jobs = [executor.submit(_forward, parameter)\
                for parameter in parameters]
print(time.time()-start)

In [ ]:
## Collect the results
outputs = []
for job in jobs:
    outputs.append(job.result())
#     print(job.result())

#### Store simulations

In [ ]:
print("fraction of blown up simulations:", sum(np.array(outputs) > 0) / len(outputs))

In [ ]:
save_data_to_hdf5(save_dir + round_name + ".h5",
                  thetas=thetas,
                  seeds=seeds,
                  spiketimes_workdir=simulator_params["workdir"],
                  con_type=simulator_params["con_type"],
                  outputs=outputs,
                  params=simulator_params)

Congratulations, you have now simulated your rules in spiking networks and stored the raw weight traces and spike trains.
Now head over to Compute_metrics to compute start the analysis of your simulations